## How far do I live from a Hospital?

### In this notebook we will put the hospitals in Mexico City on a map and group them based on their distance in order to better contemplate which area of ​​the City we should live in if we need timely access to a Hospital.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium==0.5.0
import folium # map rendering library


In [3]:
neighborhood_latitude, neighborhood_longitude =19.424468, -99.137616

In [4]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Hospitals within 30 km around

In [5]:
QUERY='hospital'

In [6]:
LIMIT =  1000 

radius = 30000 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}&categoryId=4bf58dd8d48988d196941735'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    QUERY)

In [7]:
results = requests.get(url).json()

## Obtaining hospitals using the Foursquare API

In [8]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Hospital De Cardiología CM SXXI,Hospital,19.408854,-99.151785
1,Asociación Para Evitar La Ceguera En México,Hospital,19.349050,-99.155807
2,Centro de Cáncer,Hospital,19.399834,-99.203202
3,Hospital Ángeles Mocel,Hospital,19.410990,-99.184817
4,Centro Médico Nacional Siglo XXI,Hospital,19.407822,-99.154191


In [9]:
latitude, longitude =19.424468, -99.137616

## Creating a map of the data obtained

In [10]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, nearby_venue in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories'], nearby_venues['name']):
    label = '{}, {}'.format(nearby_venue, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Clustering data

In [11]:
k=5
cluster = nearby_venues.drop(['categories','name'],1)

In [12]:
kmeans = KMeans(n_clusters = k,random_state=0).fit(cluster)
kmeans.labels_
nearby_venues.insert(0, 'Cluster Labels', kmeans.labels_)

In [13]:
# create map
map_clusters = folium.Map(location=[latitude, longitude],zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [14]:
nearby_venues.sort_values()

,Cluster Labels,name,categories,lat,lng
0,0,Hospital De Cardiología CM SXXI,Hospital,19.408854,-99.151785
1,1,Asociación Para Evitar La Ceguera En México,Hospital,19.349050,-99.155807
2,4,Centro de Cáncer,Hospital,19.399834,-99.203202
3,4,Hospital Ángeles Mocel,Hospital,19.410990,-99.184817
4,0,Centro Médico Nacional Siglo XXI,Hospital,19.407822,-99.154191
5,1,Instituto Nacional de Pediatría,Hospital,19.306288,-99.185407
6,4,Centro Médico ABC,Hospital,19.356620,-99.282241
7,0,Hospital Nuestra Señora de la Luz,Hospital,19.434879,-99.156732
8,4,Centro Médico ABC,Hospital,19.400374,-99.203726
9,2,Centro Quirurgico Río Consulado,Medical Center,19.463335,-99.152248


In [18]:
nearby_venues.groupby(['Cluster Labels']).agg(['count'])[('name', 'count')]

Cluster Labels
0    48
1    15
2    18
3     4
4    15
Name: (name, count), dtype: int64

### Cluster 0: Red 48
### Cluster 1: Purple 15
### Cluster 2: Blue 18
### Cluster 3: Green 4
### Cluster 4: Orange 15
